In [140]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime

# PostgreSQL connection config
conn = psycopg2.connect(
    dbname="5310_project",
    user="postgres",
    password="123",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Extract with Error Handling
try:
    df = pd.read_excel('Dataset_5310.xlsx')
    print("Data extraction successful.")
except Exception as e:
    print(f"Error reading Excel file: {e}")
    raise


Data extraction successful.


In [142]:
# Standardize numerical fields
# (Excel already reads numerical fields correctly; no additional conversion needed.)

# Standardize date fields
date_fields = ['Date_post', 'Transaction_date', 'Appointment_Date', 'Start_date']
for col in date_fields:
    df[col] = pd.to_datetime(df[col], errors='coerce')  
    # Convert columns to datetime format; if conversion fails, assign NaT (Not a Time).

# Standardize boolean fields (Attendance)
df['Attendence'] = df['Attendence'].apply(
    lambda x: 1 if str(x).strip().lower() in ['yes', 'true', '1'] else 0
)
# Map values such as 'yes', 'true', '1' to 1; others to 0.

# Standardize email fields
email_fields = ['Buyer_Email', 'Seller_Email', 'Employees_email']
for col in email_fields:
    df[col] = df[col].str.strip().str.lower()
# Remove leading/trailing spaces and convert email addresses to lowercase.

print("Ready!")

Ready!


In [144]:
# Fill missing Broker Title values with 'Individual Broker' to ensure data completeness.
df['BROKERTITLE'] = df['BROKERTITLE'].fillna('Individual Broker')
print("Ready!")

Ready!


In [146]:
# Basic Data Profiling (Raw Data Understanding)
print("Missing values per column:")
print(df.isnull().sum())

print("Data types per column:")
print(df.dtypes)

Missing values per column:
BROKERTITLE                       0
TYPE                              0
PRICE                             0
BEDS                              0
BATH                              0
PROPERTYSQFT                      0
ADDRESS                           0
Neighborhood/Boroughs             0
State                             0
Zipcode                           0
ADMINISTRATIVE_AREA_LEVEL_2       0
LOCALITY                          0
SUBLOCALITY                       0
STREET_NAME                       0
LONG_NAME                         0
FORMATTED_ADDRESS                 0
LATITUDE                          0
LONGITUDE                         0
Buyer_FirstNames                  0
Buyer_LastName                    0
Buyer_Email                       0
Buyer_Phone                       0
Buyer_STREET                      0
Buyer_UNIT                     3902
Buyer_CITY                        0
Buyer_DISTRICT                    0
Buyer_REGION                      0
B

In [148]:
# Necessary Business Field Validation
# Since all primary key IDs are auto-generated (SERIAL), we validate only important business fields.

# Properties Table Validation
assert df['PRICE'].notnull().all(), "Missing PRICE value!"
assert (df['PRICE'] > 0).all(), "PRICE must be greater than 0!"
assert df['TYPE'].notnull().all(), "Missing property TYPE!"
assert df['ADDRESS'].notnull().all(), "Missing property ADDRESS!"
assert df['Date_post'].notnull().all(), "Missing property posting date!"
assert df['Property_status'].notnull().all(), "Missing property status!"

# Transactions Table Validation
assert df['Transaction_date'].notnull().all(), "Missing transaction date!"
assert df['Payment_status'].notnull().all(), "Missing payment status!"

print("All necessary property and transaction-related fields validated successfully.")

All necessary property and transaction-related fields validated successfully.


In [150]:
import psycopg2

def create_tables():
    # Connect to PostgreSQL
    conn = psycopg2.connect(
    dbname="5310_project",
    user="postgres",
    password="123",
    host="localhost",
    port="5432"
    )
    cursor = conn.cursor()

    table_queries = [
    # Addresses
    """
    CREATE TABLE IF NOT EXISTS addresses (
        address_id SERIAL PRIMARY KEY,
        street_name TEXT,
        sublocality TEXT,
        locality TEXT,
        admin_area_level_2 TEXT,
        state TEXT,
        zipcode TEXT,
        latitude NUMERIC,
        longitude NUMERIC,
        formatted_address TEXT UNIQUE
    )
    """,

    # Buyers
    """
    CREATE TABLE IF NOT EXISTS buyers (
        buyer_id SERIAL PRIMARY KEY,
        name TEXT,
        email TEXT UNIQUE,
        phone TEXT,
        address_id INT REFERENCES addresses(address_id),
        unit TEXT,
        city TEXT,
        district TEXT,
        region TEXT,
        zipcode TEXT,
        budget NUMERIC
    )
    """,

    # Sellers
    """
    CREATE TABLE IF NOT EXISTS sellers (
        seller_id SERIAL PRIMARY KEY,
        name TEXT,
        email TEXT UNIQUE,
        phone TEXT,
        address_id INT REFERENCES addresses(address_id),
        unit TEXT,
        city TEXT,
        district TEXT,
        region TEXT,
        zipcode TEXT
    )
    """,

    # Employees
    """
    CREATE TABLE IF NOT EXISTS employees (
        employee_id SERIAL PRIMARY KEY,
        name TEXT,
        phone TEXT,
        email TEXT UNIQUE
    )
    """,

    # Offices
    """
    CREATE TABLE IF NOT EXISTS offices (
        office_id SERIAL PRIMARY KEY,
        title TEXT,
        address_id INT REFERENCES addresses(address_id),
        UNIQUE (title, address_id)
    )
    """,

    # Properties
    """
    CREATE TABLE IF NOT EXISTS properties (
        property_id SERIAL PRIMARY KEY,
        office_id INT REFERENCES offices(office_id),
        type TEXT,
        price NUMERIC,
        beds INT,
        bath INT,
        sqft NUMERIC,
        address_id INT REFERENCES addresses(address_id),
        status TEXT,
        date_post DATE
    )
    """,

    # Transactions
    """
    CREATE TABLE IF NOT EXISTS transactions (
        transaction_id SERIAL PRIMARY KEY,
        property_id INT REFERENCES properties(property_id),
        buyer_id INT REFERENCES buyers(buyer_id),
        seller_id INT REFERENCES sellers(seller_id),
        transaction_date DATE,
        payment_status TEXT
    )
    """,

    # Appointments
    """
    CREATE TABLE IF NOT EXISTS appointments (
        appointment_id SERIAL PRIMARY KEY,
        property_id INT REFERENCES properties(property_id),
        employee_id INT REFERENCES employees(employee_id),
        appointment_date DATE,
        appointment_time TIME,
        attendance BOOLEAN
    )
    """,

    # Performances
    """
    CREATE TABLE IF NOT EXISTS performances (
        performance_id SERIAL PRIMARY KEY,
        employee_id INT REFERENCES employees(employee_id),
        start_date DATE,
        position TEXT,
        salary NUMERIC,
        commission NUMERIC
    )
    """
]


    try:
        for query in table_queries:
            cursor.execute(query)
        conn.commit()
        print("Ready!")  
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()  
    finally:
        cursor.close()
        conn.close()

# Call the function to create tables
create_tables()


Ready!


In [152]:
for _, row in df.iterrows():
    # Insert into addresses
    address_data = (
        row['STREET_NAME'], 
        row['SUBLOCALITY'], 
        row['LOCALITY'], 
        row['ADMINISTRATIVE_AREA_LEVEL_2'], 
        row['State'], 
        row['Zipcode'], 
        row['LATITUDE'], 
        row['LONGITUDE'], 
        row['FORMATTED_ADDRESS']
    )
    
    cursor.execute("""
        INSERT INTO addresses (street_name, sublocality, locality, admin_area_level_2, state, zipcode, latitude, longitude, formatted_address)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (formatted_address) DO NOTHING
    """, address_data)

    cursor.execute("""
        SELECT address_id FROM addresses WHERE formatted_address = %s
    """, (row['FORMATTED_ADDRESS'],))
    address_id = cursor.fetchone()[0]

    # Insert into buyers
    buyer_data = (
        row['Buyer_FirstNames'] + ' ' + row['Buyer_LastName'],
        row['Buyer_Email'],
        row['Buyer_Phone'],
        address_id,
        row['Buyer_UNIT'],
        row['Buyer_CITY'],
        row['Buyer_DISTRICT'],
        row['Buyer_REGION'],
        row['Buyer_Zipcode'],
        row['Buyer_budget']
    )

    cursor.execute("""
        INSERT INTO buyers (name, email, phone, address_id, unit, city, district, region, zipcode, budget)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (email) DO NOTHING
    """, buyer_data)

    cursor.execute("""
        SELECT buyer_id FROM buyers WHERE email = %s
    """, (row['Buyer_Email'],))
    buyer_id = cursor.fetchone()[0]

    # Insert into sellers
    seller_data = (
        row['Seller_FirstNames'] + ' ' + row['Seller_LastNames'],
        row['Seller_Email'],
        row['Seller_Phone'],
        address_id,
        row['Seller_UNIT'],
        row['Seller_CITY'],
        row['Seller_DISTRICT'],
        row['Seller_REGION'],
        row['Seller_Zipcode']
    )

    cursor.execute("""
        INSERT INTO sellers (name, email, phone, address_id, unit, city, district, region, zipcode)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (email) DO NOTHING
    """, seller_data)

    cursor.execute("""
        SELECT seller_id FROM sellers WHERE email = %s
    """, (row['Seller_Email'],))
    seller_id = cursor.fetchone()[0]

    # Insert into employees
    employee_data = (
        row['Employee_Firstnames'] + ' ' + row['Employee_Lastnames'],
        row['Employees_phone'],
        row['Employees_email']
    )

    cursor.execute("""
        INSERT INTO employees (name, phone, email)
        VALUES (%s, %s, %s)
        ON CONFLICT (email) DO NOTHING
    """, employee_data)

    cursor.execute("""
        SELECT employee_id FROM employees WHERE email = %s
    """, (row['Employees_email'],))
    employee_id = cursor.fetchone()[0]

    # Insert into offices
    office_data = (
        row['BROKERTITLE'],
        address_id
    )

    cursor.execute("""
        INSERT INTO offices (title, address_id)
        VALUES (%s, %s)
        ON CONFLICT (title, address_id) DO NOTHING
    """, office_data)

    cursor.execute("""
        SELECT office_id FROM offices WHERE title = %s AND address_id = %s
    """, (row['BROKERTITLE'], address_id))
    office_id = cursor.fetchone()[0]

    # Insert into properties
    property_data = (
        office_id,
        row['TYPE'],
        row['PRICE'],
        row['BEDS'],
        row['BATH'],
        row['PROPERTYSQFT'],
        address_id,
        row['Property_status'],
        row['Date_post']
    )

    cursor.execute("""
        INSERT INTO properties (office_id, type, price, beds, bath, sqft, address_id, status, date_post)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        RETURNING property_id
    """, property_data)

    property_id = cursor.fetchone()[0]

    # Insert into transactions
    transaction_data = (
        property_id,
        buyer_id,
        seller_id,
        row['Transaction_date'],
        row['Payment_status']
    )

    cursor.execute("""
        INSERT INTO transactions (property_id, buyer_id, seller_id, transaction_date, payment_status)
        VALUES (%s, %s, %s, %s, %s)
        RETURNING transaction_id
    """, transaction_data)

    transaction_id = cursor.fetchone()[0]

    # Insert into appointments
    appointment_data = (
        property_id,
        employee_id,
        row['Appointment_Date'],
        row['Appointment_Time'],
        row['Attendence']
    )

    cursor.execute("""
        INSERT INTO appointments (property_id, employee_id, appointment_date, appointment_time, attendance)
        VALUES (%s, %s, %s, %s, %s::BOOLEAN)
        RETURNING appointment_id
    """, appointment_data)

    # Insert into performances
    performance_data = (
        employee_id,
        row['Start_date'],
        row['Position'],
        row['Salary'],
        row['Commission']
    )

    cursor.execute("""
        INSERT INTO performances (employee_id, start_date, position, salary, commission)
        VALUES (%s, %s, %s, %s, %s)
    """, performance_data)

print("Ready! Data successfully inserted.")

Ready! Data successfully inserted.


In [88]:
# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Data inserted successfully.")

Data inserted successfully.
